In [1]:
import tensorflow as tf

import keras
from keras.models import model_from_json

In [2]:
with  open('nude_weights.299x299.json', 'r') as json_file:
        loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json,custom_objects= {"GlorotUniform": tf.keras.initializers.glorot_uniform})

In [4]:
loaded_model.load_weights('nude_weights.299x299.h5')
print("Loaded model from disk")

Loaded model from disk


In [5]:
from tensorflow..saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

model_version = '1'
export_dir = 'export/Servo/' + model_version

In [25]:
import shutil
shutil.rmtree(export_dir)

In [21]:
builder = tf.compat.v1.saved_model.Builder(export_dir)

In [22]:
if tf.executing_eagerly():
   tf.compat.v1.disable_eager_execution()

In [23]:
signature = predict_signature_def(
      inputs={"inputs": loaded_model.input}
    , outputs={"score": loaded_model.output})

In [24]:
import tensorflow.python.keras.backend as K
with K.get_session() as sess:

    builder.add_meta_graph_and_variables(
          sess=sess
        , tags=[tag_constants.SERVING]
        , signature_def_map={"serving_default": signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1\saved_model.pb


In [26]:
from keras import backend as K
from keras.models import load_model
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Function to export Keras model to Protocol Buffer format
# Inputs:
#       path_to_h5: Path to Keras h5 model
#       export_path: Path to store Protocol Buffer model

def export_h5_to_pb(path_to_h5, export_path):

    # Set the learning phase to Test since the model is already trained.
    K.set_learning_phase(0)

    # Load the Keras model
    keras_model = load_model(path_to_h5)

    # Build the Protocol Buffer SavedModel at 'export_path'
    builder = saved_model_builder.SavedModelBuilder(export_path)

    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(inputs={"images": keras_model.input},
                                      outputs={"scores": keras_model.output})

    with K.get_session() as sess:
        # Save the meta graph and the variables
        builder.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING],
                                         signature_def_map={"predict": signature})

    builder.save()

In [38]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

In [40]:
with  open('nude_weights.299x299.json', 'r') as json_file:
        loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json,custom_objects= {"GlorotUniform": tf.keras.initializers.glorot_uniform})

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_385:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\backend.py", line 3601, in batch_set_value
    get_session().run(assign_ops, feed_dict=feed_dict)  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\backend.py", line 630, in get_session
    _initialize_variables(session)  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\backend.py", line 1052, in _initialize_variables
    is_initialized = session.run(  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\backend.py", line 1053, in <listcomp>
    [variables_module.is_variable_initialized(v) for v in candidate_vars])  File "C:\ProgramData\Anaconda3\lib\site-packages\te

In [39]:
predict_single_image(r'C:\\Users\\guilo\\mba-tcc\\data\\test\\safest\\safest_3426.jpg')

ValueError: Your Layer or Model is in an invalid state. This can happen if you are interleaving estimator/non-estimator models or interleaving models/layers made in tf.compat.v1.Graph.as_default() with models/layers created outside of it. Converting a model to an estimator (via model_to_estimator) invalidates all models/layers made before the conversion (even if they were not the model converted to an estimator). Similarly, making a layer or a model inside a a tf.compat.v1.Graph invalidates all layers/models you previously made outside of the graph.

In [34]:
def predict_single_image(image_path):
    
    image_data = image.load_img(image_path, target_size=(299, 299))
    image_array = image.img_to_array(image_data)
    image_array /= 255

    plt.imshow(image_array);
    plt.axis('off');

    x_train = []
    x_train.append(image_array)
    x_test = np.array(x_train)

    predictions = loaded_model.predict(x_test)
    print(f'Not Safe: {round(predictions[0][0]*100, 2)}%')
    print(f'Safe: {round(predictions[0][1]*100, 2)}%')